In [1]:
import tensorflow as tf
from sqlalchemy import create_engine
import sqlalchemy as db
import pandas as pd
import numpy as np
from tensorflow.python.client import device_lib
from keras.models import load_model
import gc
import onnx
import onnxruntime as rt

from mclLib.featurenames import *
from mclLib.cleaner import *
from mclLib.scaler import *
from mclLib.ftploader import *
from mclLib.modelpostfix import *
from mclLib.tester import *
from mclLib.server_info import *

In [2]:
print(device_lib.list_local_devices())

engine = create_engine(mysql_server_uri)
conn = engine.connect()

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 15358273372371113772
xla_global_id: -1
]


In [3]:
''' get db data '''
br_full_data = pd.read_sql_table('buyreports', conn)

In [4]:
''' db data filter '''
get_filter = ( br_full_data['isAllBuyed'] == 1) & ( br_full_data['isAllSelled'] == 1) & (br_full_data['dTradeTime'] >= '2023-02-24')
br = br_full_data[get_filter]

In [5]:
''' get features name'''
feature_names =  f_name_102
feature_size = len(feature_names)

''' set X data '''
X = br[feature_names].to_numpy(dtype=np.float64)

''' set y data '''
y_condition = (br['fMaxPowerAfterBuyWhile30'] < 0.025)
y = np.where(y_condition, 1, 0)

y_test =y
y_predict = []

modelTester = ModelTester(engine, conn)

In [14]:
# 테스트할 모델

scale_yes_model_names = ['fMax10_40_Standard_ac_max']

scale_no_model_names = ['rf_maxM_d3_v1', 'rf_maxM_d3_v2', 'rf_maxM_d2_v2', 'rf_maxM_d45_v2', 'rf_max30_d4_v2']


In [15]:
ftp= FtpLoader(ftp_ip, ftp_port, ftp_id, ftp_pw)

In [16]:

for idx, mn in enumerate(scale_yes_model_names):
    ftp.download(mn+onnx_, onnx_path, just_onnx)

    x_datas = modelTester.setFullScale(X, mn+onnx_)
    
    sess = rt.InferenceSession(onnx_path + mn + onnx_)
    res = sess.run(['output'],{'input': x_datas})
    y_predict.append(res[0])
    
    del sess
    gc.collect()

In [17]:
for idx, mn in enumerate(scale_no_model_names):
    ftp.download(mn+onnx_, onnx_path, just_onnx)
    
    x_datas = X

    # onnx_model = onnx.load(onnx_path + mn + onnx_)
    sess = rt.InferenceSession(onnx_path + mn + onnx_)
    res = sess.run(['output'],{'input': x_datas})
    y_predict.append(res[0])
    
    del sess
    gc.collect()

In [18]:
#################################
############## test #############

# suc_crit = 0.8 # 1이라 판정할 값의 기준
# fl_crit = 0.5  # 0이라 판정할 값의 기준

suc_ratio = 0.65 # 모델들에서 1이라 종합할 비율
fail_ratio = 0.65 # 모델들에서 0이라 종합할 비율

testClassification(y_test, y_predict, suc_pass_ratio=suc_ratio, fail_pass_ratio=fail_ratio)


총량 :  88658
0 :  25340 , 비율 :  28.58174107243565 (%)
1 :  63318 , 비율 :  71.41825892756435 (%)

============ predict 0 =============
총 횟수 :  560 ,  타겟기준 :  0.0
실제 0 :  356
실제 1 :  204
정답비율 :  63.57142857142857 (%)

============ predict 1 =============
총 횟수 :  61515 , 타겟기준 :  1.0
실제 1 :  49460
실제 0 :  12055
정답비율 :  80.40315370234902 (%)



In [19]:
removeTestData(onnx_list=scale_yes_model_names+scale_no_model_names)